# Analysis

### 1. Loading the project

In [ ]:
# basic imports from brightway
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
from bw2io.importers import SingleOutputEcospold2Importer
import bw2analyzer as bwa
from bw2data import methods

# other relevant packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

09:14:08+0200 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


In [2]:
# define a project where we install the databases and work in this script
bd.projects.set_current('brightway25')

### 2. Database overview

In [ ]:
bd.databases

Databases dictionary with 2 object(s):
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-consequential

In [12]:
bd.projects.current

'brightway25'

In [4]:
ei_clca = bd.Database('ecoinvent-3.10-consequential')
ei_bio = bd.Database('ecoinvent-3.10-biosphere')


### 3. Database exploration

In [25]:
ei_clca.search('smartphone')

['consumer electronics production, mobile device, smartphone' (unit, GLO, None),
 'treatment of used smartphone, mechanical treatment' (kilogram, GLO, None),
 'market for power adapter, for smartphone' (unit, GLO, None),
 'market for consumer electronics, mobile device, smartphone' (unit, GLO, None),
 'market for used smartphone' (kilogram, GLO, None),
 'power adapter production, for smartphone' (unit, GLO, None),
 'electronic component production, passive, mobile, earpiece and speaker' (kilogram, GLO, None),
 'liquid crystal display production, unmounted, mobile device' (kilogram, GLO, None),
 'consumer electronics production, mobile device, tablet' (unit, GLO, None),
 'printed wiring board production, mounted mainboard, mobile device, double-sided, Pb free' (square meter, GLO, None)]

In [22]:
# define a process as object in the project
aluminium = ei_clca.get(name = 'consumer electronics production, mobile device, smartphone', location = "GLO", unit = 'unit')


### 4. Impact assessment methods

In [ ]:
lcia_gwp100 = ('EF v3.1', 'climate change', 'global warming potential (GWP100)')
lcia_water = ('EF v3.1','water use','user deprivation potential (deprivation-weighted water consumption)')
lcia_land = ('EF v3.1', 'land use', 'soil quality index')